## mount drive

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [18]:
import os
import shutil
import os
import numpy as np
import matplotlib.pyplot as plt

import zipfile

import tensorflow as tf
#import tensorflow_hub as hub
import math
import itertools
from sklearn.metrics import classification_report, confusion_matrix
import shutil
from keras.regularizers import l2


## unzib currency data partions and comine them into training and validition

In [6]:
os.chdir("/content")

# Create a ZipFile Object and load sample.zip in it\
from zipfile import ZipFile 
with ZipFile('/content/drive/My Drive/Currency_Data/Valid_NewX.zip', 'r') as zipObj:
# Extract all the contents of zip file in current directory
    zipObj.extractall()

In [7]:
# Create a ZipFile Object and load sample.zip in it\
from zipfile import ZipFile 
with ZipFile('/content/drive/My Drive/Currency_Data/Curr_DataْX_5', 'r') as zipObj:
# Extract all the contents of zip file in current directory
    zipObj.extractall()


In [8]:
  
os.mkdir('/content/10')

os.chdir("/content/10")
# Create a ZipFile Object and load sample.zip in it\
from zipfile import ZipFile 
with ZipFile('/content/drive/My Drive/Currency_Data/Curr_DataْX_10.zip', 'r') as zipObj:
# Extract all the contents of zip file in current directory
    zipObj.extractall()


In [9]:
os.mkdir('/content/20')

os.chdir("/content/20")
# Create a ZipFile Object and load sample.zip in it\
from zipfile import ZipFile 
with ZipFile('/content/drive/My Drive/Currency_Data/Curr_DataْX_20.zip', 'r') as zipObj:
# Extract all the contents of zip file in current directory
    zipObj.extractall()


In [ ]:
os.mkdir('/content/50')

os.chdir("/content/50")
# Create a ZipFile Object and load sample.zip in it\
from zipfile import ZipFile 
with ZipFile('/content/drive/My Drive/Currency_Data/Curr_DataْX_50.zip', 'r') as zipObj:
# Extract all the contents of zip file in current directory
    zipObj.extractall()


In [11]:
os.mkdir('/content/100')

os.chdir("/content/100")
# Create a ZipFile Object and load sample.zip in it\
from zipfile import ZipFile 
with ZipFile('/content/drive/My Drive/Currency_Data/Curr_DataْX_100.zip', 'r') as zipObj:
# Extract all the contents of zip file in current directory
    zipObj.extractall()


In [12]:
os.mkdir('/content/200')

os.chdir("/content/200")
# Create a ZipFile Object and load sample.zip in it\
from zipfile import ZipFile 
with ZipFile('/content/drive/My Drive/Currency_Data/Curr_DataْX_200.zip', 'r') as zipObj:
# Extract all the contents of zip file in current directory
    zipObj.extractall()


In [15]:
os.mkdir('/content/EG_curr_data/')
os.mkdir('/content/EG_curr_data/train')


shutil.move('/content/content/Data/5B', '/content/EG_curr_data/train')
shutil.move('/content/content/Data/5F', '/content/EG_curr_data/train')

shutil.move('/content/10/content/Data/10B', '/content/EG_curr_data/train')
shutil.move('/content/10/content/Data/10F', '/content/EG_curr_data/train')

shutil.move('/content/20/content/Data/20B', '/content/EG_curr_data/train')
shutil.move('/content/20/content/Data/20F', '/content/EG_curr_data/train')

shutil.move('/content/50/content/Data/50B', '/content/EG_curr_data/train')
shutil.move('/content/50/content/Data/50F', '/content/EG_curr_data/train')

shutil.move('/content/100/content/Data/100B', '/content/EG_curr_data/train')
shutil.move('/content/100/content/Data/100F', '/content/EG_curr_data/train')

shutil.move('/content/200/content/Data/200B', '/content/EG_curr_data/train')
shutil.move('/content/200/content/Data/200F', '/content/EG_curr_data/train')

shutil.move("/content/content/drive/My Drive/Currency_data+new/valid","/content/EG_curr_data/valid")

'/content/EG_curr_data/valid'

##Model

In [ ]:
base_dir = '/content/EG_curr_data'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'valid')

batch_size = 128
epochs = 64
IMG_SHAPE = 350 # Our training data will consists of images with width of 150 pixels and height of 150 pixels


In [ ]:
pre_model = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(include_top=False, weights='imagenet',input_shape=(350, 350, 3),classes=12)
pre_model.summary()



Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 350, 350, 3) 0                                            
__________________________________________________________________________________________________
conv2d_1015 (Conv2D)            (None, 174, 174, 32) 864         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1015 (Batch (None, 174, 174, 32) 96          conv2d_1015[0][0]                
__________________________________________________________________________________________________
activation_1015 (Activation)    (None, 174, 174, 32) 0           batch_normalization_1015[0][0]   
________________________________________________________________________________

In [ ]:
for layer in pre_model.layers[:-4]:
    layer.trainable = False


In [ ]:
# train_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                    rescale=1./255, 
                    rotation_range=13, 
                    width_shift_range=.12, 
                    height_shift_range=.12, 
                    horizontal_flip=False, 
                    zoom_range=0.4
                    )

validation_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255) # Generator for our validation data

train_data_gen = train_image_generator.flow_from_directory(
                                                batch_size=batch_size, 
                                                directory=train_dir, 
                                                shuffle=True, 
                                                class_mode='categorical',
                                                target_size=(IMG_SHAPE,IMG_SHAPE))

val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size, 
                                                              directory=validation_dir, 
                                                              class_mode='categorical',
                                                              target_size=(IMG_SHAPE,IMG_SHAPE),shuffle=True) #(224,224)



Found 155100 images belonging to 12 classes.
Found 19515 images belonging to 12 classes.


In [ ]:

model_fine = tf.keras.models.Sequential()

model_fine.add(pre_model)
model_fine.add(tf.keras.layers.LayerNormalization(axis=1 , center=True , scale=True))
model_fine.add(tf.keras.layers.Flatten())

#model_fine.add(tf.keras.layers.Dense(64, activation='relu',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.005, l2=0.01)))

#model_fine.add(tf.keras.layers.Dropout(0.15))
#model_fine.add(tf.keras.layers.Dense(32, activation='relu',kernel_regularizer=l2(0.01)))
model_fine.add(tf.keras.layers.Dropout(0.2))
model_fine.add(tf.keras.layers.Dense(12, activation='softmax'))

model_fine.summary()
#pre_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Model)  (None, 9, 9, 1536)        54336736  
_________________________________________________________________
layer_normalization_3 (Layer (None, 9, 9, 1536)        18        
_________________________________________________________________
flatten_3 (Flatten)          (None, 124416)            0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 124416)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 12)                1493004   
Total params: 55,829,758
Trainable params: 4,689,438
Non-trainable params: 51,140,320
_________________________________________________________________


In [ ]:
model_fine.compile(optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
history = model_fine.fit_generator(train_data_gen, validation_data=val_data_gen, epochs=10, steps_per_epoch=20, validation_steps=20)


Epoch 1/10
20/20 [==============================] - 155s 8s/step - loss: 3.2581 - categorical_accuracy: 0.0984 - val_loss: 2.8895 - val_categorical_accuracy: 0.1500
Epoch 2/10
20/20 [==============================] - 154s 8s/step - loss: 2.9767 - categorical_accuracy: 0.1453 - val_loss: 2.7056 - val_categorical_accuracy: 0.1887
Epoch 3/10
20/20 [==============================] - 158s 8s/step - loss: 2.8201 - categorical_accuracy: 0.1945 - val_loss: 2.5815 - val_categorical_accuracy: 0.2313
Epoch 4/10
20/20 [==============================] - 158s 8s/step - loss: 2.7155 - categorical_accuracy: 0.2211 - val_loss: 2.4792 - val_categorical_accuracy: 0.2684
Epoch 5/10
20/20 [==============================] - 152s 8s/step - loss: 2.6472 - categorical_accuracy: 0.2477 - val_loss: 2.4640 - val_categorical_accuracy: 0.2758
Epoch 6/10
20/20 [==============================] - 153s 8s/step - loss: 2.5360 - categorical_accuracy: 0.2742 - val_loss: 2.3632 - val_categorical_accuracy: 0.3145
Epoch 7/10

In [ ]:
learning_rates = [0.00005,0.00001,0.000007,0.000003,0.000001]
for lr in learning_rates:
    model_fine.compile(optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=lr), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    history = model_fine.fit_generator(train_data_gen, validation_data=val_data_gen, epochs=10, steps_per_epoch=20, validation_steps=20)
    print ("\t\t\t******************* \t\t",lr," \t\t******************* \t\t \n \n")

Epoch 1/10
20/20 [==============================] - 250s 12s/step - loss: 2.7178 - categorical_accuracy: 0.3402 - val_loss: 2.4949 - val_categorical_accuracy: 0.3984
Epoch 2/10
20/20 [==============================] - 242s 12s/step - loss: 2.5462 - categorical_accuracy: 0.3805 - val_loss: 2.5073 - val_categorical_accuracy: 0.4082
Epoch 3/10
 1/20 [>.............................] - ETA: 0s - loss: 2.5500 - categorical_accuracy: 0.3750

KeyboardInterrupt: ignored

In [ ]:
learning_rates = [0.0000005,0.0000001]
for lr in learning_rates:
    model_fine.compile(optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=lr), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    history = model_fine.fit_generator(train_data_gen, validation_data=val_data_gen, epochs=15, steps_per_epoch=20, validation_steps=20)
    print ("\t\t\t******************* \t\t",lr," \t\t******************* \t\t \n \n")

Epoch 1/15
20/20 [==============================] - 49s 2s/step - loss: 1.4485 - categorical_accuracy: 0.6898 - val_loss: 1.3930 - val_categorical_accuracy: 0.7023
Epoch 2/15
20/20 [==============================] - 46s 2s/step - loss: 1.3641 - categorical_accuracy: 0.7180 - val_loss: 1.3642 - val_categorical_accuracy: 0.7078
Epoch 3/15
20/20 [==============================] - 44s 2s/step - loss: 1.4025 - categorical_accuracy: 0.7125 - val_loss: 1.4306 - val_categorical_accuracy: 0.6992
Epoch 4/15
20/20 [==============================] - 46s 2s/step - loss: 1.4335 - categorical_accuracy: 0.6867 - val_loss: 1.3879 - val_categorical_accuracy: 0.7086
Epoch 5/15
20/20 [==============================] - 45s 2s/step - loss: 1.3737 - categorical_accuracy: 0.7234 - val_loss: 1.4151 - val_categorical_accuracy: 0.6922
Epoch 6/15
20/20 [==============================] - 46s 2s/step - loss: 1.4079 - categorical_accuracy: 0.7070 - val_loss: 1.3904 - val_categorical_accuracy: 0.6977
Epoch 7/15
20/20

In [ ]:

model_fine = tf.keras.models.Sequential()

model_fine.add(pre_model)
model_fine.add(tf.keras.layers.LayerNormalization(axis=1 , center=True , scale=True))
model_fine.add(tf.keras.layers.Flatten())

model_fine.add(tf.keras.layers.Dense(128, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model_fine.add(tf.keras.layers.Dropout(0.25))
model_fine.add(tf.keras.layers.Dense(32, activation='relu',kernel_regularizer=tf.keras.regularizers.l1(0.01)))
model_fine.add(tf.keras.layers.Dropout(0.15))
model_fine.add(tf.keras.layers.Dense(12, activation='softmax'))

model_fine.summary()
#pre_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Model)  (None, 8, 8, 1536)        54336736  
_________________________________________________________________
layer_normalization_8 (Layer (None, 8, 8, 1536)        16        
_________________________________________________________________
flatten_8 (Flatten)          (None, 98304)             0         
_________________________________________________________________
dense_17 (Dense)             (None, 128)               12583040  
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 32)                4128      
_________________________________________________________________
dropout_11 (Dropout)         (None, 32)               

In [ ]:
learning_rates = [0.001,0.0007,0.0003,0.0001,0.00007,0.00003,0.00001,0.000007,0.000003,0.000001,0.0000005,0.0000001]
for lr in learning_rates:
    model_fine.compile(optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=lr), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    history = model_fine.fit_generator(train_data_gen, validation_data=val_data_gen, epochs=10, steps_per_epoch=20, validation_steps=20)
    print ("\t\t\t******************* \t\t",lr," \t\t******************* \t\t \n \n")

Epoch 1/10
20/20 [==============================] - 52s 3s/step - loss: 10.7651 - categorical_accuracy: 0.1992 - val_loss: 11.2721 - val_categorical_accuracy: 0.2188
Epoch 2/10
20/20 [==============================] - 45s 2s/step - loss: 11.4963 - categorical_accuracy: 0.1898 - val_loss: 11.0244 - val_categorical_accuracy: 0.2445
Epoch 3/10
20/20 [==============================] - 46s 2s/step - loss: 10.6406 - categorical_accuracy: 0.2234 - val_loss: 9.7113 - val_categorical_accuracy: 0.2922
Epoch 4/10
20/20 [==============================] - 44s 2s/step - loss: 9.3570 - categorical_accuracy: 0.2340 - val_loss: 8.6478 - val_categorical_accuracy: 0.3383
Epoch 5/10
20/20 [==============================] - 47s 2s/step - loss: 8.3197 - categorical_accuracy: 0.2547 - val_loss: 7.6023 - val_categorical_accuracy: 0.3711
Epoch 6/10
20/20 [==============================] - 45s 2s/step - loss: 7.5030 - categorical_accuracy: 0.2977 - val_loss: 6.9107 - val_categorical_accuracy: 0.4141
Epoch 7/10


KeyboardInterrupt: ignored

In [ ]:
pre_model = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(include_top=False, weights='imagenet',input_shape=(299, 299, 3),classes=12)
pre_model.summary()

for layer in pre_model.layers[:-1]:
    layer.trainable = False


Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_406 (Conv2D)             (None, 149, 149, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_406 (BatchN (None, 149, 149, 32) 96          conv2d_406[0][0]                 
__________________________________________________________________________________________________
activation_406 (Activation)     (None, 149, 149, 32) 0           batch_normalization_406[0][0]    
________________________________________________________________________________

In [ ]:
learning_rates = [0.001,0.0003,0.0001,0.00003,0.00001,0.000003,0.000001]
for lr in learning_rates:
  model_fine.compile(optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=lr), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
  history = model_fine.fit_generator(train_data_gen, validation_data=val_data_gen, epochs=6, steps_per_epoch=20, validation_steps=20)


Epoch 1/6
20/20 [==============================] - 91s 5s/step - loss: 3.8179 - categorical_accuracy: 0.4945 - val_loss: 3.3714 - val_categorical_accuracy: 0.6322
Epoch 2/6
20/20 [==============================] - 94s 5s/step - loss: 2.6773 - categorical_accuracy: 0.6855 - val_loss: 2.9659 - val_categorical_accuracy: 0.6656
Epoch 3/6
20/20 [==============================] - 95s 5s/step - loss: 2.1915 - categorical_accuracy: 0.7176 - val_loss: 2.4433 - val_categorical_accuracy: 0.6949
Epoch 4/6
20/20 [==============================] - 94s 5s/step - loss: 1.9128 - categorical_accuracy: 0.7402 - val_loss: 2.5042 - val_categorical_accuracy: 0.6832
Epoch 5/6
20/20 [==============================] - 96s 5s/step - loss: 1.7495 - categorical_accuracy: 0.7613 - val_loss: 2.3186 - val_categorical_accuracy: 0.6895
Epoch 6/6
20/20 [==============================] - 96s 5s/step - loss: 1.6751 - categorical_accuracy: 0.7668 - val_loss: 2.4008 - val_categorical_accuracy: 0.6633
Epoch 1/6
20/20 [=====

KeyboardInterrupt: ignored

In [ ]:

model_fine = tf.keras.models.Sequential()

model_fine.add(pre_model)
model_fine.add(tf.keras.layers.LayerNormalization(axis=1 , center=True , scale=True))
model_fine.add(tf.keras.layers.Flatten())

model_fine.add(tf.keras.layers.Dense(128, activation='relu',kernel_regularizer=l2(0.01)))
model_fine.add(tf.keras.layers.Dropout(0.4))
model_fine.add(tf.keras.layers.Dense(32, activation='relu',kernel_regularizer=l2(0.01)))
model_fine.add(tf.keras.layers.Dropout(0.3))
model_fine.add(tf.keras.layers.Dense(12, activation='softmax'))

model_fine.summary()
#pre_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 7, 7, 512)         20024384  
_________________________________________________________________
layer_normalization_5 (Layer (None, 7, 7, 512)         14        
_________________________________________________________________
flatten_5 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_14 (Dense)             (None, 128)               3211392   
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 32)                4128      
_________________________________________________________________
dropout_7 (Dropout)          (None, 32)               

In [ ]:
learning_rates = [0.001,0.0003,0.0001,0.00003,0.00001,0.000003,0.000001]
for lr in learning_rates:
  model_fine.compile(optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=lr), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
  history = model_fine.fit_generator(train_data_gen, validation_data=val_data_gen, epochs=5, steps_per_epoch=20, validation_steps=20)


Epoch 1/5
20/20 [==============================] - 95s 5s/step - loss: 5.4854 - categorical_accuracy: 0.5238 - val_loss: 4.4034 - val_categorical_accuracy: 0.7055
Epoch 2/5
20/20 [==============================] - 93s 5s/step - loss: 3.6170 - categorical_accuracy: 0.6871 - val_loss: 3.5548 - val_categorical_accuracy: 0.6898
Epoch 3/5
20/20 [==============================] - 92s 5s/step - loss: 2.7057 - categorical_accuracy: 0.7410 - val_loss: 2.8967 - val_categorical_accuracy: 0.7021
Epoch 4/5
20/20 [==============================] - 95s 5s/step - loss: 2.2270 - categorical_accuracy: 0.7676 - val_loss: 2.6882 - val_categorical_accuracy: 0.6938
Epoch 5/5
20/20 [==============================] - 97s 5s/step - loss: 1.9125 - categorical_accuracy: 0.7949 - val_loss: 2.5586 - val_categorical_accuracy: 0.6785
Epoch 1/5
20/20 [==============================] - 93s 5s/step - loss: 1.6557 - categorical_accuracy: 0.7973 - val_loss: 2.3537 - val_categorical_accuracy: 0.7063
Epoch 2/5
20/20 [=====

KeyboardInterrupt: ignored

In [ ]:
for layer in pre_model.layers[:-2]:
    layer.trainable = False


In [ ]:

model_fine = tf.keras.models.Sequential()

model_fine.add(pre_model)
model_fine.add(tf.keras.layers.LayerNormalization(axis=1 , center=True , scale=True))
model_fine.add(tf.keras.layers.Flatten())

model_fine.add(tf.keras.layers.Dense(64, activation='relu',kernel_regularizer=l2(0.01)
))
#model_fine.add(tf.keras.layers.Dropout(0.2))
model_fine.add(tf.keras.layers.Dense(64, activation='relu',kernel_regularizer=l2(0.01)
))
#model_fine.add(tf.keras.layers.Dropout(0.2))
model_fine.add(tf.keras.layers.Dense(12, activation='softmax'))

model_fine.summary()
#pre_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Model)  (None, 5, 5, 1536)        54336736  
_________________________________________________________________
layer_normalization_1 (Layer (None, 5, 5, 1536)        10        
_________________________________________________________________
flatten_3 (Flatten)          (None, 38400)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 64)                2457664   
_________________________________________________________________
dense_10 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_11 (Dense)             (None, 12)                780       
Total params: 56,799,350
Trainable params: 6,592,950
Non-trainable params: 50,206,400
__________________________________

bb

In [ ]:
#learning_rates = [0.003,0.001,0.0003,0.0001,0.00003,0.00001,0.000003,0.000001]
#for lr in learning_rates:
model_fine.compile(optimizer=tf.compat.v1.train.AdamOptimizer(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
history = model_fine.fit_generator(train_data_gen, validation_data=val_data_gen, epochs=50, steps_per_epoch=20, validation_steps=20)


Epoch 1/50
20/20 [==============================] - 77s 4s/step - loss: 4.7864 - categorical_accuracy: 0.1730 - val_loss: 4.6147 - val_categorical_accuracy: 0.2227
Epoch 2/50
20/20 [==============================] - 75s 4s/step - loss: 4.3323 - categorical_accuracy: 0.2496 - val_loss: 4.1092 - val_categorical_accuracy: 0.2934
Epoch 3/50
20/20 [==============================] - 75s 4s/step - loss: 3.7739 - categorical_accuracy: 0.3539 - val_loss: 3.7311 - val_categorical_accuracy: 0.3418
Epoch 4/50
20/20 [==============================] - 69s 3s/step - loss: 3.2963 - categorical_accuracy: 0.4035 - val_loss: 3.4026 - val_categorical_accuracy: 0.3443
Epoch 5/50
20/20 [==============================] - 77s 4s/step - loss: 2.9346 - categorical_accuracy: 0.4512 - val_loss: 3.1498 - val_categorical_accuracy: 0.4422
Epoch 6/50
20/20 [==============================] - 78s 4s/step - loss: 2.7364 - categorical_accuracy: 0.4715 - val_loss: 2.8742 - val_categorical_accuracy: 0.4414
Epoch 7/50
20/20

In [ ]:

model_fine = tf.keras.models.Sequential()

model_fine.add(pre_model)
model_fine.add(tf.keras.layers.LayerNormalization(axis=1 , center=True , scale=True))
model_fine.add(tf.keras.layers.Flatten())

model_fine.add(tf.keras.layers.Dense(64, activation='relu',kernel_regularizer=l2(0.01)))
model_fine.add(tf.keras.layers.Dropout(0.2))
#model_fine.add(tf.keras.layers.Dense(32, activation='relu',kernel_regularizer=l2(0.01)))
#model_fine.add(tf.keras.layers.Dropout(0.2))
model_fine.add(tf.keras.layers.Dense(12, activation='softmax'))

model_fine.summary()
#pre_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
layer_normalization_5 (Layer (None, 7, 7, 512)         14        
_________________________________________________________________
flatten_5 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                1605696   
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 12)                780       
Total params: 16,321,178
Trainable params: 3,966,298
Non-trainable params: 12,354,880
__________________________________

In [ ]:
learning_rates = [0.003,0.001,0.0003,0.0001,0.00003,0.00001,0.000003,0.000001,0.0000003]
for lr in learning_rates:
       model_fine.compile(optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=lr), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
       history = model_fine.fit_generator(train_data_gen, validation_data=val_data_gen, epochs=7, steps_per_epoch=20, validation_steps=20)


Epoch 1/7
20/20 [==============================] - 80s 4s/step - loss: 4.3046 - categorical_accuracy: 0.2914 - val_loss: 3.6882 - val_categorical_accuracy: 0.4152
Epoch 2/7
20/20 [==============================] - 82s 4s/step - loss: 3.3213 - categorical_accuracy: 0.4473 - val_loss: 2.8861 - val_categorical_accuracy: 0.5348
Epoch 3/7
20/20 [==============================] - 77s 4s/step - loss: 2.6224 - categorical_accuracy: 0.5070 - val_loss: 2.4650 - val_categorical_accuracy: 0.5551
Epoch 4/7
20/20 [==============================] - 83s 4s/step - loss: 2.2427 - categorical_accuracy: 0.5617 - val_loss: 2.1711 - val_categorical_accuracy: 0.6066
Epoch 5/7
20/20 [==============================] - 80s 4s/step - loss: 2.0417 - categorical_accuracy: 0.5766 - val_loss: 2.1111 - val_categorical_accuracy: 0.6109
Epoch 6/7
20/20 [==============================] - 81s 4s/step - loss: 1.9389 - categorical_accuracy: 0.6082 - val_loss: 2.1253 - val_categorical_accuracy: 0.5871
Epoch 7/7
20/20 [=====

KeyboardInterrupt: ignored

In [ ]:
for layer in pre_model.layers[:-4]:
    layer.trainable = False


In [ ]:
learning_rates = [0.003,0.001,0.0003,0.0001,0.00003,0.00001,0.000003,0.000001]
for lr in learning_rates:
       model_fine.compile(optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=lr), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
       history = model_fine.fit_generator(train_data_gen, validation_data=val_data_gen, epochs=7, steps_per_epoch=20, validation_steps=20)


Epoch 1/7
20/20 [==============================] - 85s 4s/step - loss: 2.8096 - categorical_accuracy: 0.3113 - val_loss: 2.2366 - val_categorical_accuracy: 0.5277
Epoch 2/7
20/20 [==============================] - 81s 4s/step - loss: 2.2490 - categorical_accuracy: 0.4734 - val_loss: 1.9622 - val_categorical_accuracy: 0.5980
Epoch 3/7
20/20 [==============================] - 86s 4s/step - loss: 2.0786 - categorical_accuracy: 0.5137 - val_loss: 2.0500 - val_categorical_accuracy: 0.5695
Epoch 4/7
20/20 [==============================] - 83s 4s/step - loss: 1.9842 - categorical_accuracy: 0.5367 - val_loss: 1.8835 - val_categorical_accuracy: 0.6191
Epoch 5/7
20/20 [==============================] - 84s 4s/step - loss: 1.8630 - categorical_accuracy: 0.5699 - val_loss: 1.8643 - val_categorical_accuracy: 0.6313
Epoch 6/7
20/20 [==============================] - 83s 4s/step - loss: 1.8532 - categorical_accuracy: 0.5805 - val_loss: 1.8268 - val_categorical_accuracy: 0.6380
Epoch 7/7
20/20 [=====

KeyboardInterrupt: ignored

In [ ]:
model_fine.save("drive/My Drive/GHARBcurr_2.h5")


In [ ]:
model_fine.evaluate_generator(val_data_gen)

acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(20)

plt.figure(figsize=(20, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model_fine.compile(optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=0.00005), loss='categorical_crossentropy', metrics=['categorical_accuracy'])

history = model_fine.fit_generator(train_data_gen, validation_data=val_data_gen, epochs=20, steps_per_epoch=20, validation_steps=20)


Epoch 1/20
20/20 [==============================] - 69s 3s/step - loss: 2.1159 - categorical_accuracy: 0.4648 - val_loss: 1.8783 - val_categorical_accuracy: 0.5773
Epoch 2/20
20/20 [==============================] - 68s 3s/step - loss: 2.0288 - categorical_accuracy: 0.4867 - val_loss: 1.8474 - val_categorical_accuracy: 0.5844
Epoch 3/20
20/20 [==============================] - 68s 3s/step - loss: 1.9135 - categorical_accuracy: 0.5117 - val_loss: 1.8193 - val_categorical_accuracy: 0.5902
Epoch 4/20
20/20 [==============================] - 65s 3s/step - loss: 1.8945 - categorical_accuracy: 0.5141 - val_loss: 1.7240 - val_categorical_accuracy: 0.6039
Epoch 5/20
20/20 [==============================] - 68s 3s/step - loss: 1.8430 - categorical_accuracy: 0.5383 - val_loss: 1.7429 - val_categorical_accuracy: 0.6164
Epoch 6/20
20/20 [==============================] - 70s 3s/step - loss: 1.8217 - categorical_accuracy: 0.5344 - val_loss: 1.6930 - val_categorical_accuracy: 0.6141
Epoch 7/20
20/20

In [ ]:
model_fine.evaluate_generator(val_data_gen)

acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(40)

plt.figure(figsize=(20, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
train_data_gen.class_indices

{'100B': 0,
 '100F': 1,
 '10B': 2,
 '10F': 3,
 '200B': 4,
 '200F': 5,
 '20B': 6,
 '20F': 7,
 '50B': 8,
 '50F': 9,
 '5B': 10,
 '5F': 11,
 'No_curr': 12}

In [ ]:
model=tf.keras.models.load_model("/content/drive/My Drive/GHARBcurr.h5")

In [ ]:
import cv2
img=cv2.imread("/content/drive/My Drive/Mis_predicted1/164_7.jpg")
dim=(350,350)
img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
img = np.expand_dims(img, axis=0)
img = img / 255
#img = np.array(img, dtype=np.float32)
#img = np.reshape(img, (-1, image_x, image_y, 1))
pred_probab = model.predict(img)[0]
pred_probab 

array([2.5789283e-02, 1.9854952e-03, 4.8894208e-04, 1.6032040e-02,
       1.9381220e-02, 5.6533422e-03, 1.5688317e-01, 7.5678986e-01,
       1.0659717e-02, 4.1324091e-03, 4.8998510e-04, 1.7144473e-03],
      dtype=float32)

### prediction

In [ ]:
import cv2
import tensorflow as tf
y = ['100',  '100', '10',  '10',  '200',  '200',  '20', '20',  '50',  '50',  '5', '5']
#import model
model=tf.keras.models.load_model("/content/drive/My Drive/hanycurr_84.30%.h5",compile=False)

def Curr_Pred(path):
    img=cv2.imread(path)
    dim=(350,350)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    img = np.expand_dims(img, axis=0)
    img = img / 255
    #img = np.array(img, dtype=np.float32)
    #img = np.reshape(img, (-1, image_x, image_y, 1))
    pred_probab = model.predict(img)[0]

    if max(pred_probab) < 0.45:
        print("Please take another picture")
    else:    
        pred_class = list(pred_probab).index(max(pred_probab))
        return y[list(pred_probab).index(max(pred_probab))]


#predection
predec = Curr_Pred("/content/drive/My Drive/valid/10B/1 (2).jpg")        

In [ ]:
Curr_Pred("/content/wrong_predectionXXyy213_3.jpg")

'100'

{'100B': 0,
 '100F': 1,
 '10B': 2,
 '10F': 3,
 '200B': 4,
 '200F': 5,
 '20B': 6,
 '20F': 7,
 '50B': 8,
 '50F': 9,
 '5B': 10,
 '5F': 11,
 'No_curr': 12}

# get the accurcy of our model

in our data, we considered front and back sides of the currency as diffrent classes. but practically, if we give the model back side and it predicted it as front, that woildn't be a problem.

so we need a script to get the practical accurcy:

In [ ]:
import glob
import cv2
pred_true=0
x=['5B','10B','20B','50B','100B','200B','5F','10F','20F','50F','100F','200F']
for xx in x:
    data_path = os.path.join("/content/Curr_data_13/valid/"+xx,'*g')
    files = glob.glob(data_path)
    for f1,img in enumerate(files):
        pred_class=validate_test(img)
        if (xx == '5B' or xx == '5F') and (pred_class  == 10 or pred_class == 11):
            pred_true+= 1

        elif (xx == '10B' or xx == '10F')  and (pred_class  == 2 or pred_class == 3):
            pred_true+= 1

        elif (xx == '20B' or xx == '20F')  and (pred_class  == 6 or pred_class == 7):
            pred_true+= 1

        elif (xx == '100B'  or xx == '100F')and (pred_class  == 0 or pred_class == 1):
            pred_true+= 1

        elif (xx == '200B'  or xx == '200F')and (pred_class  == 4 or pred_class == 5):
            pred_true+= 1
        elif (xx == '50B' or xx == '50F')  and (pred_class  == 8 or pred_class == 9):
            pred_true+= 1

        else:
            img=cv2.imread(img)
            cv2.imwrite('/content/Mis_predicted3/'+str(f1)+'_'+str(pred_class)+'.jpg', img)

        if f1 % 500 ==0:
          print(pred_true)            

print(pred_true)

no_curr=0


In [ ]:
validation_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255) # Generator for our validation data

val_data_gen = validation_image_generator.flow_from_directory(directory="/content/drive/My Drive/valid")


Found 5600 images belonging to 12 classes.


In [ ]:
from zipfile import ZipFile
with ZipFile('new_valid.zip', 'w') as zipObj:
   # Iterate over all the files in directory
   for folderName, subfolders, filenames in os.walk("/content/drive/My Drive/valid"):
       for filename in filenames:
           #create complete filepath of file in directory
           filePath = os.path.join(folderName, filename)
           # Add file to zip
           zipObj.write(filePath)

In [ ]:
data_path = os.path.join("/content/Mis_predicted1",'*g')
files = glob.glob(data_path)
len(files)

522

##check models

In [ ]:

import cv2
dim=(224,224)

def validate_test(path):
    img=cv2.imread(str(path)) #str(path)
    dim=(224,224)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    img = np.expand_dims(img, axis=0)
    img = img / 255
    #img = np.array(img, dtype=np.float32)
    #img = np.reshape(img, (-1, image_x, image_y, 1))
    pred_probab = model.predict(img)[0]
    return pred_probab



In [ ]:
import cv2

def validate_test(path):
    dim=(400,400)
    img=cv2.imread(str(path)) #str(path)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    img = np.expand_dims(img, axis=0)
    img = img / 255
    #img = np.array(img, dtype=np.float32)
    #img = np.reshape(img, (-1, image_x, image_y, 1))
    pred_probab = model.predict(img)[0]
    return pred_probab
pred_false_true=0
very_false=0
pred_true=0
Models= ['/content/drive/My Drive/hanycurr_84.30%.h5']

n_pred_true=[]

for mod_path in Models:
    pred_true=0
    pred_false=0
    print(mod_path, '\n')
    model=tf.keras.models.load_model(mod_path,compile=False)
    
    for xx in x:
        data_path = os.path.join("/content/content/drive/My Drive/valid/"+xx,'*g')
        files = glob.glob(data_path)
        for f1,img in enumerate(files):
            pred_probab=validate_test(img)
            pred_class = list(pred_probab).index(max(pred_probab))

            if (xx == '5B' or xx == '5F') and (pred_class  == 10 or pred_class == 11) : #and max(pred_probab) > 0.5 
                pred_true+= 1
                if max(pred_probab) > 0.5:
                    pred_false_true +=1
            elif (xx == '10B' or xx == '10F')  and (pred_class  == 2 or pred_class == 3):
                pred_true+= 1
                if max(pred_probab) > 0.5:
                    pred_false_true +=1
                

            elif (xx == '20B' or xx == '20F')  and (pred_class  == 6 or pred_class == 7):
                pred_true+= 1
                if max(pred_probab) > 0.5:
                    pred_false_true +=1

            elif (xx == '100B'  or xx == '100F')and (pred_class  == 0 or pred_class == 1):
                pred_true+= 1
                if max(pred_probab) > 0.5:
                    pred_false_true +=1

            elif (xx == '200B'  or xx == '200F')and (pred_class  == 4 or pred_class == 5):
                pred_true+= 1
                if max(pred_probab) > 0.5:
                    pred_false_true +=1

            elif (xx == '50B' or xx == '50F')  and (pred_class  == 8 or pred_class == 9):
                pred_true+= 1
                if max(pred_probab) > 0.5:
                    pred_false_true +=1

            else:

                if max(pred_probab) > 0.5:
                    pred_false+=1
                    
                else:
                   very_false+=1
                


        print(len(files),"\t",pred_true,"\t",pred_false_true,"\t",pred_false,"\t",very_false,)                
    n_pred_true.append(pred_true)
    print(n_pred_true, pred_false)
        


/content/drive/My Drive/hanycurr_84.30%.h5 

444 	 429 	 424 	 11 	 4
431 	 835 	 827 	 30 	 10
335 	 1126 	 1111 	 63 	 21
413 	 1479 	 1448 	 106 	 38
430 	 1876 	 1841 	 136 	 41
511 	 2363 	 2319 	 149 	 52
492 	 2791 	 2736 	 195 	 70
440 	 3193 	 3127 	 217 	 86
372 	 3524 	 3441 	 242 	 102
445 	 3864 	 3762 	 316 	 133
467 	 4285 	 4170 	 339 	 156
526 	 4752 	 4625 	 377 	 177
[4752] 377
